In [ ]:
Un problème de vp ???
#image_2019-06-15_10-47-26_1112_64_2.jpg

In [1]:
#D'abord vérifier si on a du Lenet

In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import time
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model, preprocessing 
from keras.applications.imagenet_utils import preprocess_input
from sklearn import metrics

from numpy.random import seed
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline


Using TensorFlow backend.


In [2]:
def build_model (width, height, depth, classes):
	# initialize the input shape and channels dimension to be
	# "channels last" ordering
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = Sequential([
		# CONV => RELU => BN => POOL layer set
		Conv2D(16, (3, 3), padding="same", input_shape=inputShape),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 2 => POOL layer set
		Conv2D(32, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(32, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 3 => POOL layer set
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# first (and only) set of FC => RELU layers
		Flatten(),
		Dense(256),
		Activation("relu"),
		BatchNormalization(),
		Dropout(0.5),
		# softmax classifier
		Dense(classes),
		Activation("softmax")
	])
	# return the built model to the calling function
	return model

In [3]:
#Paramètres

#generateur_path='/mnt/VegaSlowDataDisk/c3po/Chaine_de_traitement/Train_imagettes_annotées/type_oiseau/Materiel/generateur.csv'
test_size=0.2


epochs=200
batch_size = 600
zoom_range = 1.25
horizontal_flip = True
Minimum_Number_Class=100
dropout_rate=0.3
#steps_per_epoch=len(data_train)//batch_size
steps_per_epoch=1
#validation_steps=len(data_test)//batch_size
validation_steps=1

In [4]:

base_img_paths="/home/marcpozzo/Desktop/c3po/Images_aquises/"
generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur_bigger.csv'

df=pd.read_csv(generateur_path)
df.drop('labels',inplace=True,axis=1)

df["class"].unique()
for c in df:
    print(''+c+':',len(df[c].unique()))
df["class"].unique()


All_Unique=df["class"].unique()
Utilisable=[]
for i in df["class"].unique():
    if df["class"][df["class"]==i].count()>Minimum_Number_Class:
        Utilisable.append(i)
Utilisable
Non_Utilisable=set(All_Unique)-set(Utilisable)
Non_Utilisable
for i in Non_Utilisable:
    df=df[df["class"]!=i]
df=df[df["class"]!="oiseau"]  
df["class"].unique()


for i in range(len(df["class"])):
    image_name=df["img_paths"].iloc[i]
    df["img_paths"].iloc[i]=os.path.join(base_img_paths,image_name)
    

class: 13
img_paths: 6220


In [5]:
n_path="/mnt/VegaSlowDataDisk/c3po_interface/bin/fp_images/"



fp_new=pd.read_csv("/mnt/VegaSlowDataDisk/c3po_interface/bin/fp_threshold/table_fp_new.csv")

liste_img_paths=[]
for i in range(len(fp_new)):
    liste_img_paths.append(n_path+fp_new["imagetteName"].iloc[i])

fp_new["img_paths"]=liste_img_paths
#imagetteNamefp_new.head()

fp_new["class"]="autre"
fp_new.columns

to_drop=['path', 'filename', 'imagetteName', 'max_cat', 'cat', 'xmin', 'xmax','ymin', 'ymax', 'former_index']

fp_new.drop(to_drop,axis=1,inplace=True)

df=pd.concat([df,fp_new])

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:

seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator()

test_data_generator = ImageDataGenerator()
    #preprocessing_function = preprocess_input)




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))

gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]

Found 40827 validated image filenames belonging to 6 classes.
Found 10207 validated image filenames belonging to 6 classes.


In [7]:
val_acc=[]
acc=[]
val_loss=[]
loss=[]

seed(1)
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))
y_predict=model.predict(x_test).argmax(axis=1)
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
val_loss.append(history.history["val_loss"])
loss.append(history.history["loss"])
print(metrics.classification_report(y_predict,y_test,digits=5))

seed(1)
tf.random.set_seed(2)
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))
y_predict=model.predict(x_test).argmax(axis=1)
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
val_loss.append(history.history["val_loss"])
loss.append(history.history["loss"])
print(metrics.classification_report(y_predict,y_test,digits=5))


seed(1)
tf.random.set_seed(2)
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))
y_predict=model.predict(x_test).argmax(axis=1)
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
val_loss.append(history.history["val_loss"])
loss.append(history.history["loss"])
print(metrics.classification_report(y_predict,y_test,digits=5))


seed(1)
tf.random.set_seed(2)
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))
y_predict=model.predict(x_test).argmax(axis=1)
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
val_loss.append(history.history["val_loss"])
loss.append(history.history["loss"])
print(metrics.classification_report(y_predict,y_test,digits=5))

seed(1)
tf.random.set_seed(2)
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))
y_predict=model.predict(x_test).argmax(axis=1)
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
val_loss.append(history.history["val_loss"])
loss.append(history.history["loss"])
print(metrics.classification_report(y_predict,y_test,digits=5))



model.save("m_min_fp")

Train on 40827 samples, validate on 10207 samples
Epoch 1/10
40827/40827 [==============================] - 39s 948us/sample - loss: 0.3675 - acc: 0.9053 - val_loss: 0.2370 - val_acc: 0.9540
Epoch 2/10
40827/40827 [==============================] - 37s 912us/sample - loss: 0.1399 - acc: 0.9622 - val_loss: 0.1656 - val_acc: 0.9607
Epoch 3/10
40827/40827 [==============================] - 34s 839us/sample - loss: 0.1108 - acc: 0.9688 - val_loss: 0.1305 - val_acc: 0.9622
Epoch 4/10
40827/40827 [==============================] - 35s 851us/sample - loss: 0.0911 - acc: 0.9732 - val_loss: 0.1231 - val_acc: 0.9681
Epoch 5/10
40827/40827 [==============================] - 34s 822us/sample - loss: 0.0825 - acc: 0.9764 - val_loss: 0.0804 - val_acc: 0.9778
Epoch 6/10
40827/40827 [==============================] - 33s 817us/sample - loss: 0.0701 - acc: 0.9788 - val_loss: 0.0638 - val_acc: 0.9812
Epoch 7/10
40827/40827 [==============================] - 33s 818us/sample - loss: 0.0622 - acc: 0.9810 